# 01 — Quickstart (DuckDB over Parquet)

In [ ]:
import duckdb, pandas as pd, matplotlib.pyplot as plt, os
PARQUET_DIR='parquet'

In [ ]:
con = duckdb.connect()
con.execute(f"CREATE VIEW blocks AS SELECT * FROM parquet_scan('{PARQUET_DIR}/blocks/*.parquet');")
con.execute(f"CREATE VIEW txs AS SELECT * FROM parquet_scan('{PARQUET_DIR}/transactions/*.parquet');")
con.execute('PRAGMA threads=%d' % os.cpu_count())

In [ ]:
# Daily tx counts
q = con.execute("""
SELECT date_trunc('day', timestamp) AS day, count(*) AS tx_count
FROM blocks JOIN txs ON txs.block_number = blocks.number
GROUP BY day ORDER BY day
""").fetchdf()
q.head()

In [ ]:
q.plot(x='day', y='tx_count', kind='bar', figsize=(10,3)); plt.title('Daily tx counts'); plt.tight_layout(); plt.show()

In [ ]:
# Top senders
con.execute("SELECT \"from\" AS addr, count(*) AS sent FROM txs GROUP BY addr ORDER BY sent DESC LIMIT 10").fetchdf()